In [160]:
# Basic imports
import scipy, pandas as pd, numpy as np
from matplotlib import pyplot as plt

# set the constants
THRESHOLDS = [-6.67, -1.11, 4.44, 26.67, 32.22, 37.78]

In [161]:
# reshape, remove unnecessary columns and add datetime indices
def proc(frame, year, type):
    if (type == 'min'):
        label = "tmin"
    elif (type == 'max'):
        label = "tmax"
    else:
        return -1

    processed = frame.copy()
    for name in list(frame):
        if name[:4] != (label) and name != "GEOID20":
            processed = processed.drop(name, axis=1)
    
    processed = processed.melt(id_vars='GEOID20', var_name='Date', value_name=label)
    
    # conversion to Python datetime objects
    processed['Date'] = processed['Date'].map(lambda x: x.lstrip(label)) + str(year)
    processed['Date'] = pd.to_datetime(processed['Date'], format='%b%d%Y')

    # Set + sort by multi-level index using 'GEOID20' and 'Date'
    processed.set_index(['GEOID20', 'Date'], inplace=True)
    processed.sort_index(inplace=True)

    return processed

In [162]:
def combine(minFrame, maxFrame, year):
    pminframe = proc(minFrame, year, 'min')
    pmaxframe = proc(maxFrame, year, 'max')

    avgframe = pd.DataFrame(index=pminframe.index)
    avgframe.loc[:, 'tavg'] = (pminframe['tmin'] + pmaxframe['tmax']) / 2

    return avgframe

In [163]:
def bin(avgframe):
    zips = avgframe.index.get_level_values('GEOID20')
    
    thresholds = THRESHOLDS.copy()
    length = len(thresholds)
    threshold_counts = []

    # first threshold (strict less than)

    # Filter the dataframe to get only entries exceeding the threshold
    exceed_threshold_df = avgframe[avgframe['tavg'] < thresholds[0]]
    # Group by zip code and count the number of exceedances
    grouped_df = exceed_threshold_df.groupby(['GEOID20']).size().reset_index(name=f'<{thresholds[0]}')
    # Set the index to all zip codes and fill others with 0s
    grouped_df = grouped_df.set_index('GEOID20').reindex(zips.unique()).fillna(0).reset_index()
    # Convert the count values to integers
    grouped_df[f'<{thresholds[0]}'] = grouped_df[f'<{thresholds[0]}'].astype(int)
    # Store the counts for this threshold in the array
    threshold_counts.append(grouped_df)

    # loop of interval thresholds

    i = 1
    while i < (length):
        threshold_df = avgframe[(avgframe['tavg'] < thresholds[i]) & (avgframe['tavg'] >= thresholds[i-1])]
        grouped_df = threshold_df.groupby(['GEOID20']).size().reset_index(name=f'[{thresholds[i-1]},{thresholds[i]})')
        grouped_df = grouped_df.set_index('GEOID20').reindex(zips.unique()).fillna(0).reset_index()
        grouped_df[f'[{thresholds[i-1]},{thresholds[i]})'] = grouped_df[f'[{thresholds[i-1]},{thresholds[i]})'].astype(int)
        threshold_counts.append(grouped_df)
        i = i + 1
    
    # for last threshold (geq 100)

    exceed_threshold_df = avgframe[avgframe['tavg'] >= thresholds[length - 1]]
    grouped_df = exceed_threshold_df.groupby(['GEOID20']).size().reset_index(name=f'>={thresholds[length - 1]}')
    grouped_df = grouped_df.set_index('GEOID20').reindex(zips.unique()).fillna(0).reset_index()
    grouped_df[f'>={thresholds[length - 1]}'] = grouped_df[f'>={thresholds[length - 1]}'].astype(int)
    threshold_counts.append(grouped_df)

    # merge them into sorted different columns
    result_df = threshold_counts[0]
    for threshold in threshold_counts[1:]:
        result_df = pd.merge(result_df, threshold, on=['GEOID20'], how='outer')
    
    return result_df

In [164]:
# mCount section: find for each year the number of days per month various threholds are exceeded
years = range(2006, 2023)
columnNames = []

# setting column names
columnNames.append(f'<{THRESHOLDS[0]}')
i = 1
while i < len(THRESHOLDS):
    columnNames.append(f'[{THRESHOLDS[i-1]},{THRESHOLDS[i]})')
    i = i+1
columnNames.append(f'>={THRESHOLDS[len(THRESHOLDS) - 1]}')
columnNames.insert(0, 'GEOID20')

In [168]:
import csv

for year in years:
    minframe = pd.read_csv("extracted_by_year/" + str(year) + "_extracted/tmin" + str(year) + ".csv")
    maxframe = pd.read_csv("extracted_by_year/" + str(year) + "_extracted/tmax" + str(year) + ".csv")

    avgframe = combine(minframe, maxframe, year)
    result = bin(avgframe)

    # Convert to DataFrame
    df = pd.DataFrame(result, columns=columnNames)

    # Save to CSV with quoted column names and data
    df.to_csv("tempbins" + str(year) + ".csv", index=False, quoting=csv.QUOTE_ALL)
   #  np.savetxt("tempbins" + str(year) + ".csv", result, fmt='%s', delimiter=', ', header=', '.join(columnNames))